# Calculate velocity errors over stable surfaces (Stable Surface Error) for all velocity maps in a folder

Generates a table with all velocity map parameters, including the stable surface error.

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import datetime
import fiona
import rasterio as rio
from rasterio.mask import mask
from shapely.geometry import mapping

### Set paths to velocity maps, the stable surface masks (ssm), and a glacier shapefile (UTM projection)

In [ ]:
# path to autorift velocity maps:
autorift_outp = '/Users/jukesliu/Documents/TURNER/DATA/VELOCITY_MAPS/output_AutoRIFT/'

# set path to folder with the stable surface mask (should have been generated when running custom autoRIFT)
# contains files named ssm_#m.tif
refvpath = '/Users/jukesliu/Documents/TURNER/DATA/VELOCITY_MAPS/forAutoRIFT/'
print(os.listdir(refvpath))

# read in glacier shapefile and grab outline as shapes
shp_path = '/Users/jukesliu/Documents/TURNER/DATA/shapefiles_gis/Turner_RGI/TG_glims_polygon_UTM7.shp'
with fiona.open(shp_path) as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

### Grab important information and Stable Surface Error (SSE) from each velocity map

In [ ]:
# initialize lists to store info for each velocity map
ds1s = []; ds2s = []; dts = []; chipsizes = []; sats = []; data_fracs = []; err_vx = []; err_vy = []; err_v = []; 
file_count = 0

# for each velocity map
for output in os.listdir(autorift_outp):
    if output.startswith('velocity') and output.endswith('.tif'):
        print(output) # display file names
        [filetype, ds1, ds2, chipsize, sat] = output.split('_') # grab info from filename
        d1 = datetime.datetime.strptime(ds1, '%Y%m%d'); d2 = datetime.datetime.strptime(ds2, '%Y%m%d')
        dt = d2-d1
        sat = sat.split('.')[0] # s
        ds1s.append(ds1); ds2s.append(ds2); dts.append(dt.days)
        chipsizes.append(int(chipsize[:-1])); sats.append(sat)
        
        # read in the velocity map
        v_reader = rio.open(autorift_outp+output)
        vx = v_reader.read(1); vy = v_reader.read(2) # grab vx and vy
        v = np.sqrt(vx**2 + vy**2) # calculate vmag from vx and vy
        # raster grid value (UTM coordinates)
        x_vals = np.arange(v_reader.bounds.left, v_reader.bounds.right+1, v_reader.transform[0])
        y_vals = np.arange(v_reader.bounds.bottom, v_reader.bounds.top+1, v_reader.transform[0])
        
        # Read in stable surface mask
        ssm_reader = rio.open(refvpath+'ssm_'+chipsize+'.tif') # stable surface mask
        ssm = ssm_reader.read(1)
        if ssm.shape != vx.shape:
            ssm = ssm[:vx.shape[0],:vx.shape[1]]
        
        # grab data fraction in glacier outline
        cropped_img, cropped_transform = mask(v_reader, shapes, nodata= -3e5, crop=True)
        if cropped_img.shape[0] == 3:
            cropped_v = cropped_img[2] # grab the third band [2]
        else:
            cropped_vx = cropped_img[0]; cropped_vy = cropped_img[1]
            cropped_v = np.sqrt(cropped_vx**2 + cropped_vy**2) 
        cropped_v_filled = cropped_v.copy()
        cropped_v_filled[np.isnan(cropped_v_filled)] = 0 # fill all Nans with 0
        total_pixels = np.count_nonzero(cropped_v_filled >= 0) # count number of non-nodata values (negative)
        cropped_v[cropped_v < 0] = np.NaN # turn no data values (negative) into NaNs
        pixels_w_data = len(cropped_v[cropped_v >= 0]) # count non Nans (number of pixels with data)
        if pixels_w_data > 0:
            data_percent = int(pixels_w_data/total_pixels*100)
            data_fracs.append(data_percent)
            empty = False
        else:
            data_fracs.append(0)
            empty = True
#         plt.imshow(cropped_v); plt.colorbar(); plt.title(str(data_percent)+' %'); plt.show() # check visually
            
        # Grab off-ice velocities using stable surface mask
        ssm[ssm > 0] = 2 # turn ice into 2
        ssm[ssm == 0] = 1 # turn stable surfaces into 1
        ssm[ssm == 2] = 0 # turn ice into 0s
        ssm_masked_vx = ssm*vx; ssm_masked_vy = ssm*vy; ssm_masked_v = ssm*v 
        
        # calculate RMSE [m/d]
        err_vx.append(np.sqrt(np.nanmean(ssm_masked_vx**2))/365)
        err_vy.append(np.sqrt(np.nanmean(ssm_masked_vy**2))/365)
        err_v.append(np.sqrt(np.nanmean(ssm_masked_v**2))/365)
        
        # get ready for next iteration
        del v, vx, vy, empty
        file_count += 1
        
print(file_count, 'files found.')

In [ ]:
df = pd.DataFrame(list(zip(ds1s, ds2s, chipsizes, sats, dts, data_fracs, err_vx, err_vy, err_v)),
                 columns=['ds1','ds2','min_chip_size','sat','dt_days','data_percent',
                          'err_vx','err_vy','err_v'])
df.sort_values(by=['ds1','min_chip_size','sat'])
df

In [ ]:
# convert dates to datetime objects
df.ds1 = pd.to_datetime(df.ds1, format='%Y%m%d')
df.ds2 = pd.to_datetime(df.ds2, format='%Y%m%d')
df['mid_date'] = (df.ds2-df.ds1)/2+df.ds1
df

### Export CSV to a file

In [ ]:
outfilepath = 'TableS1_autorift_vmap_parameters_SK.csv' # enter path and name of CSV file to save to
df.to_csv(outfilepath)

## Plot errors

In [ ]:
# create subplots with common y-axis
fs = 12
a = 0.3 # control opacity
colors_dict = {'LS':'tab:blue', 'S2':'tab:orange', 'S1':'tab:green', 'PS':'tab:red'}
fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(14,8), sharey=True)

# 1) plot velocity residuals over stable surfaces
ax1.set_title("Velocity over stable surfaces", fontsize=fs)
ax1.plot(df.err_vx,df.mid_date, 'kx', alpha=a)
ax1.plot(df.err_vy,df.mid_date, 'gx', alpha=a)
ax1.set_xlim(0, 4)
ax1.set_xlabel('RMSE [m/d]', fontsize=fs); ax1.set_ylabel('Velocity map mid date', fontsize=fs);
ax1.legend(['X error','Y error'], loc='lower right', fontsize=fs)

# 2) plot speed residual of stable surfaces (scaled by max speed)
ax2.set_title("Speed error", fontsize=fs)
ax2.scatter(df.err_v, df.mid_date, c=df.sat.map(colors_dict), alpha=a) # color points by satellite (see colors dict)
ax2.set_xlim(0,4)
ax2.set_xlabel('RMSE [m/d]', fontsize=fs);

# 3) percent coverage on glacier
ax3.set_title("Data coverage")
ax3.hlines(df.mid_date, np.zeros(len(df.data_percent)),df.data_percent, color='k',alpha=a)
ax3.set_xlabel('Pixels with data on glacier [%]', fontsize=fs)
ax3.set_xlim(0,100)

# all plots
for ax in [ax1, ax2, ax3]:
    ax.tick_params(labelsize=fs)
    ax.grid()
    ax.set_ylim(datetime.datetime(2013,1,1), datetime.datetime(2022,1,1))

plt.suptitle('Summary of velocity map errors and coverage', fontsize=fs+4)
plt.show()